In [ ]:
"""
                        This code is generated by 
                        
                        Mustafa Mohammadi Gharasui 810897030
                        
                        University of Tehran
                        
                        Algorithm and Computation
                        
                        Parallel processing algorithms
"""
import numpy as np
from scipy.sparse import coo_matrix
from pyspark import SparkConf, SparkContext
import time


n = 10000
np.random.seed(1000)


indices = np.random.randint(0, 10, size=(2 * n, 2))

values = np.random.random(2 * n)

sparse_representation = np.c_[indices, values[:, None]]

np.savetxt('sparse_matrix.txt',
           sparse_representation, fmt = '%7.4f', delimiter=' ')



print('matrix of indices : \n {} \n shape of our matrix ,indices : {}'
      .format(indices, indices.shape))

print('\n \n vector of values : \n {} \n len(values) : {}'
      .format(values, len(values)))

print('\n \n new matrix of sparse_representation:\n {}'
      .format(sparse_representation))

print('\n \n {} \n lenght of matrix is : {} \n Type of matrix is : {}'
      .format(sparse_representation.shape, len(sparse_representation), 
              type(sparse_representation)))

M_sparse = coo_matrix((values, (indices.T[0], indices.T[1])), shape=(n, n))


print('\nType of M_sparse is : {} \n \n{} '.format(type(M_sparse),M_sparse))

M = M_sparse.toarray()
np.savetxt('M_Sparse.txt',M, fmt = '%7.4f')



In [ ]:
spark_start_time = time.time()
conf = SparkConf()
sc = SparkContext(conf = conf)

lines = sc.textFile('sparse_matrix.txt')

print('time is consumed up to {} \n lines: \n{} \n'
      .format(time.time()-spark_start_time, lines.take(10)))

In [ ]:
M_rdd = lines.map(lambda line: line.split()).map(lambda value: [float(i) for i in value])
# M_rdd = lines.map(lambda l: map(float, l.strip().split(' ')))
print('M_rdd.take(10): \n',M_rdd.take(10))
print('M_rdd : \n', M_rdd)


In [ ]:
print('Create random Vector to multiply against our matrix')
v_in = np.random.random(size = n)
v_in

In [ ]:
# MapReduce Operation
spark_time = time.time()
v_out_spark_raw = M_rdd\
    .map(lambda x: (x[0], v_in[int(x[1])] * x[2]))\
    .reduceByKey(lambda v1, v2: v1 + v2)\
    .sortByKey()\
    .collect()

print('spark time consumed is : {}'.format(time.time() - spark_time))
print('spark overall time is : {}'.format(time.time() - spark_start_time))
print(v_out_spark_raw)
print(len(v_out_spark_raw))

v_out_spark_raw = np.array(v_out_spark_raw)

In [ ]:
v_out_spark = np.zeros(n)
print(v_out_spark)

# print(v_out_spark_raw)
# v_out_spark[map(int, v_out_spark_raw.T[0])] = v_out_spark_raw.T[1]
# print(v_out_spark)

s_time = time.time()
v_out_numpy = M.dot(v_in)

print('Time os common Multiplication is : {}'.format(time.time() - s_time))
print(v_out_numpy)

In [ ]:
# def summation(v1, v2):
#     return v1 + v2

# summation = lambda v1, v2: v1 + v2
# summation(5,10)

